# Teoria

## Uczenie Temporal Difference
Metoda **model-free** oznacza, że agent uczy się jedynie poprzez doświadczenie, nie ma on dostępu do macierzy przejść MDP.
Metody Monte Carlo wymagają ukończenia całego epizodu (dopiero wtedy znana jest wartość $G_t$) zanim będzie można zaktualizować wartość $V(S_t)$.
Metody Temporal Difference natomiast mogą dokonać aktualizacji już po jednym kroku czasu

Najprostsza metoda TD dokonuje następującej aktualizacji:
$V(S_t) \leftarrow V(S_t)+\alpha[R_{r+1}+\gamma V(S_{t+1})-V(S_t)]$

Dzieje się to natychmiast po tranzycji do stanu $S_{t+1}$ oraz po otrzymaniu nagrody $R_{t+1}$.
Inaczej mówiąc, metoda ta aktualizuje wartość funkcji $V$ o mały krok $\alpha$ w kierunku celu (*TD target*), który jest w tym przypadku następujący $R_{r+1}+\gamma V(S_{t+1})$ ($G_t$ w przypadku MC). Taka metoda TD nazywana jest $TD(0)$.

Zauważmy, że wartość w nawiasach kwadratowych metody TD(0) jest pewnego rodzaju *błędem* (*TD error*), który mierzy różnicę pomiędzy estymowaną wartością $S_t$ oraz lepszą, nową estymacją $R_{r+1}+\gamma V(S_{t+1})$. Wiele algorytmów uczenia ze wzmocnieniem opartych jest na tej idei.

![](img/td0-backup.png)
Powyżej widzimy diagram dla metody TD(0). Wartość estymowana dla górnego stanu jest aktualizowana na podstawie jednej tranzycji do następnego stanu.

#### Zalety

- Nie wymagają modelu środowiska
- Może je zaimplementować jako algorytm *online*, w pełni inkrementowalnym stylu.

### SARSA
SARSA jest algorytmem **on-policy** i należy do grupy metod sterowania TD.

Jednak, zamiast uczyć się funkcji $V$, uczy się on funkcji $Q$ (funkcja action-value, która może być użyta przez agenta, kiedy nie ma on dostępu do macierzy przejść MDP).

Pierwszym krokiem metody on-policy jest estymacja $q_\pi(s,a)$ dla aktualnej strategii $\pi$ dla wszystkich stanów $s$ i akcji $a$. Możemy tego dokonać, wykorzystując metodę TD opisaną powyżej do uczenia się wartości $v_\pi$.

W poprzedniej sekcji rozważaliśmy tranzycję ze stanu do stanu i uczyliśmy się wartości stanów. Teraz będziemy rozpatrywać tranzycje z par stan-akcja do pary stan-akcja i będziemy się uczyć wartości $q_\pi$.

Aktualizacja wartości $Q$ jest następująca:
$Q(S_t, A_t) \leftarrow Q(S_t, A_t)+\alpha[R_{r+1}+\gamma Q(S_{t+1}, A_{t+1})-Q(S_t, A_t)]$

Uwaga: jest ona dokonywana po każdej tranzycji ze stanu **nieterminalnego**.

![](img/sarsa-backup.png)
Powyżej przedstawiony jest diagram dla algorytmu SARSA.

![](img/sarsa.png)

### Q-learning

Jedną z przełomowych metod uczenia ze wzmocnieniem była metoda off-policy *Q-learning* (również oparta na idei TD).

Formuła aktualizacji wartości $Q$:
$Q(S_t, A_t) \leftarrow Q(S_t, A_t)+\alpha[R_{r+1}+\gamma \max_a Q(S_{t+1}, a)-Q(S_t, A_t)]$

W tym przypadku uczona funkcja $Q$, bezpośrednio przybliża $q_*$, czyli optymalną funkcję action-value. Jest ona niezależna od strategii jaką kieruje się agent w środowisku (stąd off-policy)

![](img/q-learning-backup.png)
Jak widzimy z powyższego diagramu, do aktualizacji wartości $Q$ używamy $\max$ nad wszystkimi możliwymi akcjami (w SARSA używaliśmy tej samej akcji, co agent wykonał w środowisku). To pokazuje, że używamy tak naprawdę dwóch strategii: jednej do zachowania agenta oraz drugiej do aktualizacji wartości $Q$.

![](img/q-learning.png)

# Kod

In [4]:
import gym
import numpy as np

W tym zadaniu wykorzystamy środowisko [Taxi-v3](https://www.gymlibrary.dev/environments/toy_text/taxi/) z biblioteki Gym OpenAI.

Zdefiniujmy strategię $\epsilon-greedy$ do eksploracji środowiska.

In [5]:
def epsilon_greedy(Q, epsilon, n_actions, s, train=False):
    if train or np.random.rand() < epsilon:
        action = np.argmax(Q[s, :])
    else:
        action = np.random.randint(0, n_actions)
    return action

Inicjalizacja wartości $Q$.

In [6]:
def init_q_values(s, a, type="ones"):
    if type == "ones":
        return np.ones((s, a))
    elif type == "random":
        return np.random.random((s, a))
    elif type == "zeros":
        return np.zeros((s, a))

Implementacja algorytmu SARSA.

In [30]:
def sarsa(alpha, gamma, epsilon, episodes, max_steps):
    env = gym.make('Taxi-v3')
    n_states, n_actions = env.observation_space.n, env.action_space.n
    Q = init_q_values(n_states, n_actions, type="ones")
    timestep_reward = []
    for episode in range(episodes):
        total_reward = 0
        s = env.reset()
        a = epsilon_greedy(Q, epsilon, n_actions, s)
        t = 0
        while t < max_steps:
            t += 1
            s_, reward, done, info = env.step(a)
            total_reward += reward
            a_ = epsilon_greedy(Q, epsilon, n_actions, s_)
            '''
            Implementacja aktualizacji SARSA
            '''
            s, a = s_, a_
            if done:
                timestep_reward.append(total_reward)
                break
    return timestep_reward

Uruchomienie SARSA.

In [32]:
alpha = 0.4
gamma = 0.999
epsilon = 0.9
episodes = 1000
max_steps = 2500
timestep_reward = sarsa(alpha, gamma, epsilon, episodes, max_steps)
print(timestep_reward)

[-758, -713, -704, -749, -668, -596, -713, -776, -740, -335, -659, -695, -686, -637, -686, -609, -569, -686, -596, -542, -488, -542, -263, -695, -452, -272, -488, -333, -389, -227, -344, -308, -263, -308, -272, -704, -632, -272, -205, -263, -245, -389, -301, -317, -425, -263, -251, -109, -254, -299, -272, -245, -308, -209, -245, -740, -263, -176, -254, -389, -236, -263, -161, -245, -416, -254, -254, -245, -236, -127, -272, -263, -380, -49, -137, -226, -91, -140, -80, -184, -181, -299, -281, -128, -166, -11, -326, -254, -317, -245, -362, -138, 12, -245, -54, -60, -36, -155, 11, -254, -25, -425, -10, -245, -218, -149, -236, -290, -93, -86, -371, -248, -148, -118, -245, -164, -132, -189, -39, -130, -39, -263, -118, -127, -30, 11, -233, -18, -258, -299, -281, -201, -134, -197, -169, -317, -267, -245, -98, -163, -82, -213, -240, -290, -425, 8, -5, -160, -173, -218, -162, -49, -56, -71, -268, -17, -131, -290, -299, -380, -256, -18, -52, -74, -48, -20, -51, -7, -10, -135, -205, -21, -50, -18,

Implementacja algorytmu Q-learning.

In [33]:
def qlearning(alpha, gamma, epsilon, episodes, max_steps):
    env = gym.make('Taxi-v3')
    n_states, n_actions = env.observation_space.n, env.action_space.n
    Q = init_q_values(n_states, n_actions, type="ones")
    timestep_reward = []
    for episode in range(episodes):
        s = env.reset()
        a = epsilon_greedy(Q, epsilon, n_actions, s)
        t = 0
        total_reward = 0
        while t < max_steps:
            t += 1
            s_, reward, done, info = env.step(a)
            total_reward += reward
            a_ = np.argmax(Q[s_, :])
            '''
            Implementacja aktualizacji Q-learning
            '''
            s, a = s_, a_
            if done:
                timestep_reward.append(total_reward)
                break
    return timestep_reward

Uruchomienie Q-learning

In [34]:
alpha = 0.4
gamma = 0.999
epsilon = 0.9
episodes = 1000
max_steps = 2500
timestep_reward = qlearning(alpha, gamma, epsilon, episodes, max_steps)
print(timestep_reward)

[-740, -776, -731, -740, -488, -551, -93, -596, -308, -704, -776, -659, -668, -416, -668, -452, -200, -560, -668, -740, -101, -29, -200, -776, -359, -632, -308, -740, -560, -488, -668, -560, -560, -524, -158, -335, -452, -89, -411, -506, -434, -158, -236, -200, -133, -136, 6, -200, -227, -740, -86, -308, -80, -200, -181, -200, -26, -200, -200, -200, -138, -73, -200, -200, -145, -200, -200, -116, -200, -69, -42, -14, -118, 12, -200, -107, -112, -108, -102, -200, -200, -200, -200, -35, -15, -168, -96, -154, -200, -200, -200, -92, -147, -126, -33, -200, 9, -6, -45, -200, -61, -72, -176, -200, -85, -140, -86, -42, -28, -200, -46, -9, -81, -200, -8, -131, -15, -31, -102, -200, -81, -5, -163, -268, -113, -7, -21, -187, 1, -264, -135, -119, -152, -117, -85, -144, -143, -105, -126, -200, 4, -183, -257, -93, -36, -2, -476, -80, -200, -121, -35, 0, -60, -163, -18, -170, -156, -200, -113, -109, -6, 9, -127, -121, -74, -450, -434, -16, -86, -750, -239, -61, -4, -101, 5, -152, -3, 9, -219, -8, -37,

# Zadanie

1. Uzupełnij puste miejsca w kodzie dla algorytmu SARSA oraz Q-learning.
2. Napisz metodę, która będzie oceniała zachowanie agenta. Pokaż wizualnie, jak zachowuje się agent, używając wyuczonej strategii (użyj `env.render()`). Dodatkowo dokonaj ewaluacji swojego agenta na przestrzeni 100 epizodów - ile razy popełnił błąd (wykonał akcję zabronioną)? Czy można wytrenować agenta w ten sposób, aby nie popełnił błędu w żadnym z epizodów?
3. Pokaż wykres, który wizualizuje proces uczenia się agenta. Innymi słowy, pokaż całkowitą nagrodę, jaką osiągnął agent za każdy epizod uczenia. Przedstaw na jednym wykresie wyniki dla obu algorytmów.

## Materiały

- [RL Course by David Silver - Lecture 5: Model Free Control](https://www.youtube.com/watch?v=0g4j2k_Ggc4&list=PLzuuYNsE1EZAXYR4FJ75jcJseBmo4KQ9-&index=9)
- Sutton, Barto - Chapter 6